# HW7

# II. Analyzing Data

In [136]:
import pandas as pd
import numpy as np
pd.options.display.float_format = "{:,.4f}".format

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from arch import arch_model

import warnings
warnings.filterwarnings("ignore")

import importlib
import utils as ut
importlib.reload(ut)

<module 'utils' from '/Users/chuan/github/0_Quantitative_Portfolio_Manager/homework/utils.py'>

# 2. Analyzing GMO

This section utilizes data in the file, `gmo_analysis_data.xlsx`.

Examine GMO's performance. Use the risk-free rate to convert the total returns to excess returns

In [131]:
file_path = './../data/gmo_analysis_data.xlsx'

descriptions = pd.read_excel(file_path, index_col=0) # first sheet
descriptions.index.name = 'Symbol'
display(descriptions)

signals = pd.read_excel(file_path, sheet_name='signals', index_col=0)
signals.index.name = 'Date'
display(signals)

total_returns = pd.read_excel(file_path, sheet_name='returns (total)', index_col=0)
total_returns.index.name = 'Date'
display(total_returns)

risk_free_rate = pd.read_excel(file_path, sheet_name='risk-free rate', index_col=0)
risk_free_rate.index.name = 'Date'
display(risk_free_rate)

excess_returns = total_returns.sub(risk_free_rate['US3M'], axis=0)
display(excess_returns)

,Unit,Type,Description
Symbol,,,
DP,Ratio,Index,Dividend-Price Ratio of the S&P500
EP,Ratio,Index,Earnings-Price Ratio of the S&P500
US10Y,Yield,Index,10-Year Tnotes
SPY,Total Return,ETF,S&P 500
GMWAX,Total Return,Mutual Fund,GMO
RF,Total Return,Index,3-Month Tbills


,DP,EP,US10Y
Date,,,
1993-02-28,2.8200,4.4400,6.0300
1993-03-31,2.7700,4.4100,6.0300
1993-04-30,2.8200,4.4400,6.0500
1993-05-31,2.8100,4.3800,6.1600
1993-06-30,2.7900,4.3100,5.8000
...,...,...,...
2023-06-30,1.5800,3.8800,3.8100
2023-07-31,1.5300,3.7600,3.9700
2023-08-31,1.5500,3.8900,4.0900


,SPY,GMWAX
Date,,
1993-02-28,0.0107,NaN
1993-03-31,0.0224,NaN
1993-04-30,-0.0256,NaN
1993-05-31,0.0270,NaN
1993-06-30,0.0037,NaN
...,...,...
2023-06-30,0.0648,0.0398
2023-07-31,0.0327,0.0244
2023-08-31,-0.0163,-0.0210


,US3M
Date,
1993-02-28,0.0025
1993-03-31,0.0025
1993-04-30,0.0025
1993-05-31,0.0026
1993-06-30,0.0026
...,...
2023-06-30,0.0045
2023-07-31,0.0046
2023-08-31,0.0046


,SPY,GMWAX
Date,,
1993-02-28,0.0082,NaN
1993-03-31,0.0199,NaN
1993-04-30,-0.0281,NaN
1993-05-31,0.0244,NaN
1993-06-30,0.0011,NaN
...,...,...
2023-06-30,0.0603,0.0352
2023-07-31,0.0281,0.0198
2023-08-31,-0.0209,-0.0256


## 2.1 Calculate the mean, volatility, and Sharpe ratio for GMWAX. Do this for three samples:
- from inception through 2011
- 2012-present
- inception - present

In [132]:
agg_years = [(1993, 2011), (2012, 2023), (1993, 2023)]
all_stats = [
    ut.summary_statistics_annualized(excess_returns[['GMWAX']].loc[f'{start}':f'{end}'].dropna())[['Mean', 'Vol', 'Sharpe']]
    .assign(Period=f'{start}-{end}')
    .reset_index()
    .rename(columns={'index': 'Factor'})
    .set_index(['Period', 'Factor'])
    for start, end in agg_years
]

result = pd.concat(all_stats)
result

,,Mean,Vol,Sharpe
Period,Factor,,,
1993-2011,GMWAX,0.0158,0.1250,0.1266
2012-2023,GMWAX,0.0364,0.0945,0.3856
1993-2023,GMWAX,0.0249,0.1125,0.2209


### Has the mean, vol, and Sharpe changed much since the case?

The mean increased and volatility decreased during 2012-2023, showing that GMO's forecasts and subsequently thier asset allocations for GMWAX worked during this period of turmoil as compared to the previous sub-period of 1993-2011.

## 2.2 GMO believes a risk premium is compensation for a security's tendency to lose money at "bad times". For all three samples, analyze extreme scenarios by looking at -
- Min return
- 5th percentile (VaR-5th)
- Maximum  Drawdown

In [133]:
agg_years = [(1993, 2011), (2012, 2023), (1993, 2023)]
all_stats = [
    ut.summary_statistics_annualized(total_returns[['GMWAX']].loc[f'{start}':f'{end}'].dropna())[['Min', 'VaR 5%', 'Max Drawdown']]
    .assign(Period=f'{start}-{end}')
    .reset_index()
    .rename(columns={'index': 'Factor'})
    .set_index(['Period', 'Factor'])
    for start, end in agg_years
]

result = pd.concat(all_stats)
result

,,Min,VaR 5%,Max Drawdown
Period,Factor,,,
1993-2011,GMWAX,-0.1450,-0.0562,-0.3552
2012-2023,GMWAX,-0.1186,-0.0368,-0.2168
1993-2023,GMWAX,-0.1450,-0.0468,-0.3552


### a) Does GMWAX have high or low tail-risk as seen by these stats

GMWAX seems to have low tail-risk as depicted by the tail risk statistics above. 

### b) Does that vary much across the two subsamples?

The tail risk is especially low in the latter sub-period of 2012-2023. This could be a factor of two aspects, better forecasting by GMO or reduced sub-sample length leading to less market downturns.

## 2.3 For all three samples, regress excess returns of GMWAX on excess returns of SPY.

- sub1 - 1993-2011
- sub2 - 2012-2023
- sub3 - 1993-2023


In [134]:
agg_years = [(1993, 2011), (2012, 2023), (1993, 2023)]
all_stats = [
    ut.time_series_regression_annualized(excess_returns[['GMWAX']].loc[f'{start}':f'{end}'],
                                         excess_returns[['SPY']].loc[f'{start}':f'{end}'],
                                         intercept=True, annual_factor=12)['df']
    .assign(Period=f'{start}-{end}')
    .set_index(['Period'])
    .rename(columns={'SPY': 'SPY Beta'})
    for start, end in agg_years
]

result = pd.concat(all_stats)
result

,Alpha,SPY Beta,Treynor Ratio,Information Ratio,R^2
Period,,,,,
1993-2011,-0.0058,0.5396,0.0293,-0.0655,0.5071
2012-2023,-0.0327,0.5738,0.0635,-0.6972,0.7544
1993-2023,-0.0166,0.5506,0.0451,-0.2277,0.5821


### a) Report the estimated alpha, beta, and r-squared.

### b) Is GMWAX a low-beta strategy? Has that changed since the case?

GMWAX seems to have a relatively moderate beta with the market: ~50%-57%, It's market beta is not very low, we can consider it a low-beta strategy. The beta remains quite stable across both sub-samples

### c) Does GMWAX provide alpha? Has that changed across the subsamples?

GMWAX does not provide alpha in either subsample as alpha is negative.



# 3 Forecast Regressions
 This section utilizes data in the file,`gmo_analysis_data.xlsx`.

## 3.1 Consider the lagged regression, where the regressor, ($X$), is a period behind the target, ($r^{SPY}$).
$$r^{SPY}_t = \alpha^{SPY,X}+(\beta^{SPY,X})'X_{t-1}+\epsilon^{SPY,X}_t$$
 Estimate (1) and report the $R^2$, as well as the OLS estimates for $\alpha$ and $\beta$. Do this for...
- $X$ as a single regressor, the dividend-price ratio.
- $X$ as a single regressor, the earnings-price ratio.
- $X$ as three regressors, the dividend-price ratio, the earnings-price ratio, and the 10-year yield.

 For each, report the r-squared.

In [135]:
regr_dp = ut.time_series_regression_annualized(total_returns[['SPY']], signals.shift()[['DP']])['df'].rename(index={'SPY': 'DP'})
display(regr_dp)
regr_ep = ut.time_series_regression_annualized(total_returns[['SPY']], signals.shift()[['EP']])['df'].rename(index={'SPY': 'EP'})
display(regr_ep)
regr_all = ut.time_series_regression_annualized(total_returns[['SPY']], signals.shift())['df'].rename(index={'SPY': 'all'})
display(regr_all)

,Alpha,DP Beta,Treynor Ratio,Information Ratio,R^2
DP,-0.1138,0.0095,10.8516,-0.7659,0.0094


,Alpha,EP Beta,Treynor Ratio,Information Ratio,R^2
EP,-0.0739,0.0033,31.7518,-0.4975,0.0087


,Alpha,DP Beta,EP Beta,US10Y Beta,Treynor Ratio,Information Ratio,R^2
all,-0.1808,0.0080,0.0027,-0.0010,10.6073,-1.2212,0.0164


## 3.2 For each of the three regressions, let’s try to utilize the resulting forecast in a trading strategy.
- Build the forecasted SPY returns: $\hat{r}^{SPY}_{t+1}$. Note that this denotes the forecast made using $X_t$ to forecast the $(t+1)$ return.
- Set the scale of the investment in SPY equal to 100 times the forecasted value:
$
w_t = 100 \hat{r}^{SPY}_{t+1}
$
- We are not taking this scaling too seriously. We just want the  strategy  to  go  bigger  inperiods where the forecast is high and to withdraw in periods where the forecast is low, or even negative.
- Calcualte the return on this strategy:
$
r^X_{t+1} = w_tr^{SPY}_{t+1}
$

You should now have the trading strategy returns, $r^x$ for each of the forecasts. For each strategy, estimate:
- mean, volatility, Sharpe,
- max-drawdown
- market alpha
- market beta
- market Information

In [185]:
def strategy_lagged_regression(returns, factors, intercept=True, annual_factor=12):
    """
    Calculate a strategy using lagged regression over the entire period 
    by assigning weights as 100 * prediction.
        returns (pd.DataFrame): Single-column DataFrame of dependent variable (e.g., asset returns).
        factors (pd.DataFrame): DataFrame of independent variables (e.g., factors/signals).
    Returns:
        pd.DataFrame: Strategy returns with weights based on lagged regression predictions.
    """
    factors = factors.shift()
    regr = ut.time_series_regression_annualized(returns, factors, intercept, annual_factor)['regrs'][0]
    if intercept:
        factors = sm.add_constant(factors)
    # weights = 100 * (factors * regr.params).dropna().sum(axis=1)
    weights = 100 * regr.fittedvalues # prediction
    strategy_returns = (weights * returns.iloc[:, 0]).dropna()
    return pd.DataFrame(strategy_returns, columns=['-'.join(factors.columns)])

ret = strategy_lagged_regression(total_returns[['SPY']], signals, intercept=True, annual_factor=12)
ut.summary_statistics_annualized(ret)

,Mean,Vol,Sharpe,Min,Max,Skewness,Kurtosis,VaR 5%,CVaR 5%,Max Drawdown,Bottom,Peak,Recovery,Duration
const-DP-EP-US10Y,0.1251,0.1456,0.8591,-0.1710,0.2211,0.1155,4.4982,-0.0641,-0.0922,-0.5246,2009-02-28,2007-10-31,2011-04-30,1277 days


In [7]:
def calculate_and_summarize_strategy(regr, signals, total_returns, excess_returns, regr_name, column_names, multi_factor=False):
    """
    Calculate weights, strategy returns, and summary statistics for a given column.
    regr: DataFrame, regression coefficients
    signals: DataFrame, signals
    total_returns, excess_returns: DataFrame, total returns and excess returns
    regr_name: str, name of the regression
    column_names: list of str, column names to use in the regression
    multi_factor: bool, whether to use multi-factor model
    """
    if multi_factor:
        weights = 100 * ((signals.shift()[column_names] * regr.loc[regr_name, column_names]).sum(axis=1) + regr.loc[regr_name, 'Alpha'] / 12)
    else:
        weights = 100 * (signals.shift()[regr_name] * regr.loc[regr_name, column_names[0]] + regr.loc[regr_name, 'Alpha'] / 12)
    strategy_returns = pd.DataFrame((weights * total_returns['SPY']).dropna(), columns=[regr_name])
    negative_risk_premium_counts = (strategy_returns < 0).sum()

    return pd.concat([ut.summary_statistics_annualized(strategy_returns),
                      ut.time_series_regression_annualized(strategy_returns, excess_returns[['SPY']].loc[strategy_returns.index]),
                      negative_risk_premium_counts.rename('Negative Risk Premium Counts')],
                      axis=1)

In [8]:
(pd.concat([calculate_and_summarize_strategy(regr_dp, signals, total_returns, excess_returns, 'DP', ['DP']),
            calculate_and_summarize_strategy(regr_ep, signals, total_returns, excess_returns, 'EP', ['EP']),
            calculate_and_summarize_strategy(regr_all, signals, total_returns, excess_returns, 'all', ['DP', 'EP', 'US10Y'], multi_factor=True)])
            [['Mean', 'Vol', 'Sharpe', 'Max Drawdown', 'Alpha', 'SPY', 'Information Ratio']]
            .rename(columns={'Alpha': 'Market Alpha',
                             'SPY': 'Market Beta',
                             'Information Ratio': 'Market IR'}))

,Mean,Vol,Sharpe,Max Drawdown,Market Alpha,Market Beta,Market IR
DP,0.1095,0.1489,0.7359,-0.6570,0.0411,0.8617,0.5490
EP,0.1081,0.1289,0.8382,-0.3853,0.0498,0.7335,0.7326
all,0.1242,0.1455,0.8539,-0.5246,0.0624,0.7781,0.7109


## 3.3 GMO believes a risk premium is compensation for a security's tendency to lose money at "bad times". Let's consider risk characteristics.

### a) For both strategies, the market, and GMO, calculate the monthly VaR for $\pi=.05$. Just use the quantile of the historic data for this VaR calculation.

In [9]:
df_strats = (pd.concat([calculate_and_summarize_strategy(regr_dp, signals, total_returns, excess_returns, 'DP', ['DP']),
                        calculate_and_summarize_strategy(regr_ep, signals, total_returns, excess_returns, 'EP', ['EP']),
                        calculate_and_summarize_strategy(regr_all, signals, total_returns, excess_returns, 'all', ['DP', 'EP', 'US10Y'], multi_factor=True)]))
pd.concat([df_strats[['VaR 5%']], 
           ut.summary_statistics_annualized(excess_returns[['SPY']])[['VaR 5%']],
           ut.summary_statistics_annualized(excess_returns[['GMWAX']].dropna())[['VaR 5%']] ])

,VaR 5%
DP,-0.0523
EP,-0.0539
all,-0.0640
SPY,-0.0735
GMWAX,-0.0471


### b) The GMO case mentions that stocks under-performed short-term bonds from 2000-2011. Does the dynamic portfolio above under-perform the risk-free rate over this time?

All dynamic portfolios outperform the risk-free rate.

In [10]:
(pd.concat([calculate_and_summarize_strategy(regr_dp, signals['2000':'2011'], total_returns['2000':'2011'], excess_returns['2000':'2011'], 'DP', ['DP']),
            calculate_and_summarize_strategy(regr_ep, signals['2000':'2011'], total_returns['2000':'2011'], excess_returns['2000':'2011'], 'EP', ['EP']),
            calculate_and_summarize_strategy(regr_all, signals['2000':'2011'], total_returns['2000':'2011'], excess_returns['2000':'2011'], 'all', ['DP', 'EP', 'US10Y'], multi_factor=True)])
            [['Mean', 'Vol', 'Sharpe', 'Max Drawdown', 'Alpha', 'SPY', 'Information Ratio']]
            .rename(columns={'Alpha': 'Market Alpha',
                             'SPY': 'Market Beta',
                             'Information Ratio': 'Market IR'}))

,Mean,Vol,Sharpe,Max Drawdown,Market Alpha,Market Beta,Market IR
DP,0.0407,0.1866,0.2179,-0.6570,0.0410,0.9672,0.4109
EP,0.0400,0.1350,0.2961,-0.3853,0.0402,0.6190,0.4483
all,0.0665,0.1601,0.4154,-0.5246,0.0701,0.7265,0.6513


### c) Based on the regression estimates, in how many periods do we estimate a negative risk premium?

In [11]:
df_strats[['Negative Risk Premium Counts']]

,Negative Risk Premium Counts
DP,130
EP,132
all,118


### d) Do you believe the dynamic strategy takes on extra risk?

No, judging by the tail risk metrics and volatility of the dynamic strategies compared to SPY it does not seem like these strategies take on extra risk on the whole.

However, we must keep in mind that the strategies are dependent on running regressions with very little prediction power, so badly estimated parameters could lead to terrible performance. This is not evident in terms of very high volatility and tail risk in our backtesting period though.

In [12]:
df_strats[['Mean', 'Vol', 'Sharpe', 'VaR 5%', 'Max Drawdown', 'SPY', 'Alpha', 'Information Ratio']]

,Mean,Vol,Sharpe,VaR 5%,Max Drawdown,SPY,Alpha,Information Ratio
DP,0.1095,0.1489,0.7359,-0.0523,-0.6570,0.8617,0.0411,0.5490
EP,0.1081,0.1289,0.8382,-0.0539,-0.3853,0.7335,0.0498,0.7326
all,0.1242,0.1455,0.8539,-0.0640,-0.5246,0.7781,0.0624,0.7109


In [13]:
ut.summary_statistics_annualized(excess_returns[['SPY']])[['Mean', 'Vol', 'Sharpe', 'VaR 5%', 'Max Drawdown']]

,Mean,Vol,Sharpe,VaR 5%,Max Drawdown
SPY,0.0795,0.1491,0.5329,-0.0735,-0.5600


# 4. Out-of-Sample Forecasting

This section utilizes data in the file, `gmo_analysis_data.xlsx`.

Reconsider the problem above, of estimating (1) for $x$. The reported $R^2$ was the in-sample $R^2$ it examined how well the forecasts fit in the sample from which the parameters were estimated. <br><br>

**In particular, focus on the case of using both dividend-price and earnings-price as signals.**

Let's consider the out-of-sample r-squared. To do so, we need the following:
- Start at $t=60$.
- Estmiate (1) only using data through time $t$.
- Use the estimated parameters of (1), along with $x_{t+1}$ to calculate the out-of-sample forecast for the following period, $t+1$.
\begin{align}
\hat{r}^{SPY}_{t+1} = \hat{a}^{SPY,x}_t+(\beta^{SPY,x})'x_t 
\end{align}
- Calculate the $t+1$ forecast error,
\begin{align}
  e^x_{t+1} = r^{SPY}_{t+1} - \hat{r}^{SPY}_{t+1}
\end{align}
- Move to $t=61$, and loop through the rest of the sample.

You now have the time-series of out-of-sample prediction errors, $e^x$.

Calculate the time-series of out-of-sample prediction errors $e^0$, which are based on the null forecast:
\begin{align*}
\bar{r}^{SPY}_{t+1} &= \frac{1}{t}\sum^{t}_{i=1}r^{SPY}_i \\
e^0_{t+1} &= r^{SPY}_{t+1} - \bar{r}^{SPY}_{t+1}
\end{align*}


## 4.1 Report the out-of-sample $R^2$:
$$R^2_{OOS} \equiv 1-\frac{\sum^T_{i=61}(e^x_i)^2}{\sum^T_{i=61}(e^0_i)^2}$$
Note that unlike an in-sample r-squared, the out-of-sample r-squared can be anywhere between $(-\infty,1]$.

Did this forecasting strategy produce a positive OOS r-squared?

This forecasting strategy produces a negative OOS r-squared, which indicates our strategy fits the data worse than a horizontal line given by the expanding mean of the sample.


In [117]:
def expanding_regression(returns, factors, start_window=60, intercept=True):
    """Calculate expanding regression statistics for each asset against lagged factors, vectorized,
    and compare replication performance of static, is, oos against actual returns."""
    factors = factors.shift()

    oos_errors = []
    null_errors = []

    if intercept:
        factors = sm.add_constant(factors)
    for t in range(start_window, len(returns)):
        regr = sm.OLS(returns.iloc[:t], factors.iloc[:t], missing='drop').fit()
        # oos_forecast = np.dot(regr.params, factors.iloc[t])
        oos_forecast = regr.predict(factors.iloc[t])[0]
        oos_errors.append(returns.iloc[t] - oos_forecast)
        null_forecast = returns.iloc[:t].mean()
        null_errors.append(returns.iloc[t] - null_forecast)

    oos_R2 = 1 - (np.array(oos_errors)**2).sum() / (np.array(null_errors)**2).sum()
    
    return oos_R2

In [123]:
regr_oos_dp = expanding_regression(total_returns['SPY'], signals[['DP']])
regr_oos_ep = expanding_regression(total_returns['SPY'], signals[['EP']])
regr_oos_all = expanding_regression(total_returns['SPY'], signals)

results = {
    "DP": expanding_regression(total_returns['SPY'], signals[['DP']], start_window=60, intercept=True),
    "EP": expanding_regression(total_returns['SPY'], signals[['EP']], start_window=60, intercept=True),
    "DP and EP": expanding_regression(total_returns['SPY'], signals[['DP', 'EP']], start_window=60, intercept=True),
    "All Signals": expanding_regression(total_returns['SPY'], signals, start_window=60, intercept=True),
}
pd.DataFrame(list(results.items()), columns=["Signal", "OOS R-squared"]).set_index("Signal")

,OOS R-squared
Signal,
DP,-0.0021
EP,-0.0064
DP and EP,-0.0172
All Signals,-0.0307


## 4.2 Re-do problem 3.2 using this OOS forecast. 
How much better/worse is the OOS Earnings-Price ratio strategy compared to the in-sample version of 3.2?

The Out-of-Sample performs significantly worse than in-sample as expected. With lower mean, much higher volatility and tail risk measure, the oos performance seems to be taking much higher risk with a large negative beta to the market returns.

In [ ]:
df_strats = (pd.concat([calculate_and_summarize_strategy(regr_dp, signals, total_returns, excess_returns, 'DP', ['DP']),
                        calculate_and_summarize_strategy(regr_ep, signals, total_returns, excess_returns, 'EP', ['EP']),
                        calculate_and_summarize_strategy(regr_all, signals, total_returns, excess_returns, 'all', ['DP', 'EP', 'US10Y'], multi_factor=True)]))

## 4.3 Re-do problem 3.3 using this OOS forecast. 
Is the point-in-time version of the strategy riskier?

Compared to the full sub-sample, the mean returns go down significantly during 2000-2011.The volatility slightly increasesthus the strategy experiences a lower Sharpe Ratio. Given the lower performance of the strategy and worse tail risk measures compared to SPY, the strategy does take on extra risk.